In [2]:
import pandas as pd
import numpy as np

In [3]:
# 출력 옵션 변경

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 1300)

In [4]:
dlvr_raw = pd.read_csv('../data/dlvr_call.csv');

c:\users\jeaung-lee\appdata\local\programs\python\python39\lib\site-packages\IPython\core\interactiveshell.py:3169: DtypeWarning: Columns (19) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
dlvr = dlvr_raw.copy()

In [6]:
dlvr.shape

(4296271, 32)

In [7]:
dlvr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4296271 entries, 0 to 4296270
Data columns (total 32 columns):
 #   Column                     Dtype 
---  ------                     ----- 
 0   SN                         int64 
 1   PROCESS_DT                 object
 2   DLVR_REQUST_ENTRPS_ID      object
 3   DLVR_REQUST_STTUS_VALUE    int64 
 4   DLVR_RCEPT_ENTRPS_ID       object
 5   DLVR_STORE_ID              object
 6   DLVR_STORE_INDUTY_NM       object
 7   DLVR_STORE_LEGALDONG_CODE  int64 
 8   DLVR_STORE_BRTC_NM         object
 9   DLVR_STORE_SIGNGU_NM       object
 10  DLVR_STORE_LEGALDONG_NM    object
 11  DLVR_STORE_LI_NM           object
 12  DLVR_STORE_ADSTRD_CODE     int64 
 13  DLVR_STORE_RDNMADR_CODE    object
 14  DLVR_DSTN_LEGALDONG_CODE   int64 
 15  DLVR_DSTN_BRTC_NM          object
 16  DLVR_DSTN_SIGNGU_NM        object
 17  DLVR_DSTN_LEGALDONG_NM     object
 18  DLVR_DSTN_LI_NM            object
 19  DLVR_DSTN_ADSTRD_CODE      object
 20  DLVR_DSTN_RDNMADR_CODE  

In [8]:
# 지역을 서울로 한정

dlvr = dlvr[dlvr['DLVR_DSTN_BRTC_NM'] == '서울특별시']

In [9]:
# 배달이 완료된 데이터만 추출

dlvr = dlvr[dlvr['DLVR_REQUST_STTUS_VALUE'] == 1] 

In [10]:
# 지역, 업종, 매출 컬럼 선택
# 행정동코드 ['DLVR_DSTN_ADSTRD_CODE'], 배달상점 업종이름 (카테고리) ['DLVR_STORE_INDUTY_NM'], 배달상품금액 ['GOODS_AMOUNT']

df_dlvr = dlvr[['DLVR_DSTN_ADSTRD_CODE', 'DLVR_STORE_INDUTY_NM','GOODS_AMOUNT']]

In [11]:
# 행정동 코드가 이상한 행 삭제  

df_dlvr = df_dlvr[df_dlvr.DLVR_DSTN_ADSTRD_CODE != '\\N']
df_dlvr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1001280 entries, 0 to 4296270
Data columns (total 3 columns):
 #   Column                 Non-Null Count    Dtype 
---  ------                 --------------    ----- 
 0   DLVR_DSTN_ADSTRD_CODE  1001280 non-null  object
 1   DLVR_STORE_INDUTY_NM   1001280 non-null  object
 2   GOODS_AMOUNT           1001280 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 30.6+ MB


In [12]:
# 업종 컬럼 값 확인

df_dlvr['DLVR_STORE_INDUTY_NM'].unique()

array(['족발/보쌈', '패스트푸드', '분식', '치킨', '회', '아시안/양식', '한식', '피자', '돈까스/일식',
       '카페/디저트', '찜탕', '중식', '야식', '배달전문업체', '심부름', '도시락'], dtype=object)

In [13]:
# 업종이'배달전문업체','심부름'인 행 삭제

df_dlvr.replace(['배달전문업체','심부름'], np.nan, inplace=True)
df_dlvr = df_dlvr.dropna(axis=0) 

In [14]:
df_dlvr['DLVR_STORE_INDUTY_NM'].unique()

array(['족발/보쌈', '패스트푸드', '분식', '치킨', '회', '아시안/양식', '한식', '피자', '돈까스/일식',
       '카페/디저트', '찜탕', '중식', '야식', '도시락'], dtype=object)

### 지역별 업종 매출 순위 

In [15]:
area = df_dlvr['DLVR_DSTN_ADSTRD_CODE'].unique()
# area = [1159066000]
df = pd.DataFrame(columns=['DLVR_DSTN_ADSTRD_CODE','DLVR_STORE_INDUTY_NM','GOODS_AMOUNT'])

for a in area:
    df_area = df_dlvr[df_dlvr['DLVR_DSTN_ADSTRD_CODE'] == a]
    induty = df_area['DLVR_STORE_INDUTY_NM'].unique()    
    for i in induty:
        df_induty = df_area[df_area['DLVR_STORE_INDUTY_NM'] == i]
        indSum = sum(df_induty['GOODS_AMOUNT'])
        dic = { 'DLVR_DSTN_ADSTRD_CODE' : a, 'DLVR_STORE_INDUTY_NM' : i,'GOODS_AMOUNT' : indSum}
        df = df.append(dic,ignore_index=True)

In [16]:
df1 = df.copy() 

In [17]:
df1.head()

,DLVR_DSTN_ADSTRD_CODE,DLVR_STORE_INDUTY_NM,GOODS_AMOUNT
0,1159066000,족발/보쌈,13261700
1,1159066000,카페/디저트,13775500
2,1159066000,한식,7082700
3,1159066000,치킨,30202200
4,1159066000,피자,5773660


In [18]:
# 행정동코드 열 정수형으로 변경(데이터 표기 통일)

df1['DLVR_DSTN_ADSTRD_CODE'] = df1['DLVR_DSTN_ADSTRD_CODE'].astype('int64')
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3628 entries, 0 to 3627
Data columns (total 3 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   DLVR_DSTN_ADSTRD_CODE  3628 non-null   int64 
 1   DLVR_STORE_INDUTY_NM   3628 non-null   object
 2   GOODS_AMOUNT           3628 non-null   object
dtypes: int64(1), object(2)
memory usage: 85.2+ KB


In [19]:
# 상품금액 합게가 0인 행 삭제
# 상품금액 열 정수형으로 변경

df1.replace(0, np.nan, inplace=True)
df1 = df1.dropna(axis=0)
df1['GOODS_AMOUNT'] = df1['GOODS_AMOUNT'].astype('int64')
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3521 entries, 0 to 3627
Data columns (total 3 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   DLVR_DSTN_ADSTRD_CODE  3521 non-null   int64 
 1   DLVR_STORE_INDUTY_NM   3521 non-null   object
 2   GOODS_AMOUNT           3521 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 110.0+ KB


In [20]:
# area = df_dlvr['DLVR_DSTN_ADSTRD_CODE'].unique()
# # area = [1159066000]
# result = pd.DataFrame(columns=['코드','총 배달 상품 금액', '1등', '2등', '3등'])

# for a in area:
#     df_area = df1[df1['DLVR_DSTN_ADSTRD_CODE'] == a]
#     totalSum = sum(df_area['GOODS_AMOUNT'])                            #지역 총 금액 산출
#     induty = df_area['DLVR_STORE_INDUTY_NM'].unique()    
#     df_rank = pd.DataFrame(columns=['업종(금액)','배달 상품 금액'])          # 지역 내 업종별 순위를 구하기 위한 데이터 프레임
#     for i in induty:
#         df_induty = df_area[df_area['DLVR_STORE_INDUTY_NM'] == i]
#         indSum = sum(df_induty['GOODS_AMOUNT'])
#         indic = {'업종(금액)' : i + '('+ str(indSum) +'원)','배달 상품 금액' : indSum}
#         df_rank = df_rank.append(indic,ignore_index=True)
#     if len(df_rank) >= 3:
#         df_rank = df_rank.sort_values(by='배달 상품 금액', ascending=False).head(3)    
#         dic = { '코드' : a, '총 배달 상품 금액' : totalSum,'1등' : df_rank.iloc[0,0], '2등' : df_rank.iloc[1,0], '3등' : df_rank.iloc[2,0]}
    
#     else:
#         nan = {'업종(금액)' : '-' ,'배달 상품 금액' : 0 }
#         df_rank = df_rank.append(nan,ignore_index=True)
#         df_rank = df_rank.append(nan,ignore_index=True)
#         df_rank = df_rank.append(nan,ignore_index=True)
#         df_rank = df_rank.sort_values(by='배달 상품 금액', ascending=False).head(3)    
#         dic = { '코드' : a, '총 배달 상품 금액' : totalSum,'1등' : df_rank.iloc[0,0], '2등' : df_rank.iloc[1,0], '3등' : df_rank.iloc[2,0]}
    
#     result = result.append(dic,ignore_index=True)

In [37]:
area = df_dlvr['DLVR_DSTN_ADSTRD_CODE'].unique()
# area = [1159066000]
result = pd.DataFrame(columns=['코드','총 배달 상품 금액', '1등', '2등', '3등','1등 금액','2등 금액','3등 금액'])

for a in area:
    df_area = df1[df1['DLVR_DSTN_ADSTRD_CODE'] == a]
    totalSum = sum(df_area['GOODS_AMOUNT'])                            #지역 총 금액 산출
    induty = df_area['DLVR_STORE_INDUTY_NM'].unique()    
    df_rank = pd.DataFrame(columns=['업종','배달 상품 금액'])          # 지역 내 업종별 순위를 구하기 위한 데이터 프레임
    for i in induty:
        df_induty = df_area[df_area['DLVR_STORE_INDUTY_NM'] == i]
        indSum = sum(df_induty['GOODS_AMOUNT'])
        indic = {'업종' : i ,'배달 상품 금액' : indSum}
        df_rank = df_rank.append(indic,ignore_index=True)
    if len(df_rank) >= 3:
        df_rank = df_rank.sort_values(by='배달 상품 금액', ascending=False).head(3)    
        dic = { '코드' : a, '총 배달 상품 금액' : totalSum,'1등' : df_rank.iloc[0,0], '2등' : df_rank.iloc[1,0], '3등' : df_rank.iloc[2,0],
              '1등 금액' : df_rank.iloc[0,1],'2등 금액' : df_rank.iloc[1,1],'3등 금액' : df_rank.iloc[2,1]}
    
    else:
        nan = {'업종(금액)' : '-' ,'배달 상품 금액' : 0 }
        df_rank = df_rank.append(nan,ignore_index=True)
        df_rank = df_rank.append(nan,ignore_index=True)
        df_rank = df_rank.append(nan,ignore_index=True)
        df_rank = df_rank.sort_values(by='배달 상품 금액', ascending=False).head(3)    
        dic = { '코드' : a, '총 배달 상품 금액' : totalSum,'1등' : df_rank.iloc[0,0], '2등' : df_rank.iloc[1,0], '3등' : df_rank.iloc[2,0]}
    
    result = result.append(dic,ignore_index=True)

In [38]:
df3 = result[result['총 배달 상품 금액'] != 0]

In [39]:
df3

,코드,총 배달 상품 금액,1등,2등,3등,1등 금액,2등 금액,3등 금액
0,1159066000,137877970,치킨,아시안/양식,카페/디저트,43307500,20421600,20239400
1,1153079000,556014073,치킨,족발/보쌈,한식,195839051,57690201,55396600
2,1153073000,612434816,치킨,패스트푸드,분식,173461508,93529360,82218901
3,1153052000,287492495,분식,치킨,패스트푸드,69394800,48201127,34950798
4,1154562000,241159256,치킨,패스트푸드,족발/보쌈,47638100,40653895,34873300
5,1153076000,567745854,치킨,족발/보쌈,한식,145419991,98322600,62456500
6,1156065000,112364330,치킨,한식,분식,37081900,11375000,11038700
7,1129063000,6871350,카페/디저트,분식,패스트푸드,1684950,1226900,1017900
8,1153074000,883706442,치킨,패스트푸드,분식,252792436,132604110,99118152
9,1156069000,109001740,치킨,아시안/양식,족발/보쌈,51643000,10139800,8428900


### 행정동별 TOP3 업종 추출 완료.

### 행정동코드 좌표 데이터

In [23]:
geo = pd.read_excel('../data/행정_법정동 중심좌표.xlsx',header=0, index_col=False);

In [41]:
# 배달 데이터에 존재하는 지역만 추출

area = df3['코드'].unique()
dlvrArea = geo.loc[:,'코드'].isin(area) 

In [42]:
geo = geo[dlvrArea]

## 배달 데이터 + 위치정보 병합

In [50]:
df_geo_dlvr = pd.merge(df3, geo, on='코드', how='left')

In [53]:
df_geo_dlvr = df_geo_dlvr.drop('하위', axis=1)

In [55]:
df_geo_dlvr = df_geo_dlvr.dropna(axis=0)

In [56]:
df_geo_dlvr

,코드,총 배달 상품 금액,1등,2등,3등,1등 금액,2등 금액,3등 금액,시도,시군구,읍면동,위도,경도,코드종류
0,1159066000,137877970,치킨,아시안/양식,카페/디저트,43307500,20421600,20239400,서울특별시,동작구,대방동,37.506879,126.927994,H
1,1153079000,556014073,치킨,족발/보쌈,한식,195839051,57690201,55396600,서울특별시,구로구,수궁동,37.493837,126.831527,H
2,1153073000,612434816,치킨,패스트푸드,분식,173461508,93529360,82218901,서울특별시,구로구,고척제2동,37.495650,126.887771,H
3,1153052000,287492495,분식,치킨,패스트푸드,69394800,48201127,34950798,서울특별시,구로구,구로제1동,37.495650,126.887771,H
4,1154562000,241159256,치킨,패스트푸드,족발/보쌈,47638100,40653895,34873300,서울특별시,금천구,독산제2동,37.456979,126.895656,H
5,1153076000,567745854,치킨,족발/보쌈,한식,145419991,98322600,62456500,서울특별시,구로구,개봉제3동,37.495650,126.887771,H
6,1156065000,112364330,치킨,한식,분식,37081900,11375000,11038700,서울특별시,영등포구,신길제3동,37.526410,126.896252,H
7,1129063000,6871350,카페/디저트,분식,패스트푸드,1684950,1226900,1017900,서울특별시,성북구,정릉제2동,37.589440,127.016817,H
8,1153074000,883706442,치킨,패스트푸드,분식,252792436,132604110,99118152,서울특별시,구로구,개봉제1동,37.495650,126.887771,H
9,1156069000,109001740,치킨,아시안/양식,족발/보쌈,51643000,10139800,8428900,서울특별시,영등포구,신길제7동,37.526410,126.896252,H


In [57]:
df_geo_dlvr.to_csv('../data/df_geo_dlvr.csv')

In [ ]:
df_geo_dlvr